In [29]:
from pathlib import Path

import pandas as pd
import plotly.express as px


def get_df(aslprep):
    "aggregate quality control metrics across runs"
    qc_files = sorted(aslprep.glob("sub-*/ses-*/perf/*desc-quality_control_cbf.csv"))

    df = [pd.read_csv(qc_file) for qc_file in qc_files]
    map = {1: "slr + rest", 2: "supine + rest", 3: "slr + manip", 4: "supine + manip"}
    df = (
        pd.concat(df)
        .reset_index(drop=True)
        .drop(
            [
                "basilQEI",
                "pvcQEI",
                "NEG_BASIL_PERC",
                "NEG_PVC_PERC",
                "coregDC",
                "coregCC",
                "normDC",
                "normCC",
            ],
            axis=1,
        )
        .replace({"run": map})
    )

    df["run"] = df["ses"] + df["run"]

    return df.melt(
        id_vars=["sub", "run"],
        value_vars=df.iloc[:, 3:].columns,
        value_name="qc measure",
    )


"save interactive boxplot of aslprep quality control metrics"
aslprep = Path("/Volumes/projects/181814/derivatives/aslprep")
df = get_df(aslprep)

fig = px.box(
    df,
    y="qc measure",
    color="run",
    facet_col="variable",
    facet_col_wrap=6,
    height=800,
    width=1600,
    hover_data="sub",
)
fig.update_yaxes(matches=None, showticklabels=False)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.write_html(aslprep / "cbfQC.html")
